<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/Graphonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Gaoyiminggithub/Graphonomy

In [1]:
#https://stackoverflow.com/questions/53417976/import-data-from-shared-google-drive-into-google-colaboratory
#Click on the folder and then click on an option that says "Add to My Drive." The shared folder should now be visible in Google Colab.

# connect to google drive, need to copy auth code from new page
from google.colab import drive
# gdrive not working somehow
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# download repo from Github
import os
from os.path import exists, join, basename

project_name = "Graphonomy"
if not exists(project_name):
  # clone
  !git clone -q https://github.com/Gaoyiminggithub/Graphonomy.git

In [3]:
%cd Graphonomy

/content/Graphonomy


In [0]:
!pip install -r requirements

In [0]:
cihp_path = '/content/gdrive/My Drive/LIP/CIHP'
atr_path = '/content/gdrive/My Drive/LIP/ATR'
cihp_flipped_path = '/content/gdrive/My Drive/Category_rev_ids.rar'
atr_flipped_path = '/content/gdrive/My Drive/SegmentationClassAug_rev.rar'

#pretrained models
pascal_pt = '/content/gdrive/My Drive/cihp2pascal.pth'
cihp_pt = '/content/gdrive/My Drive/inference.pth'
universal_pt = '/content/gdrive/My Drive/universal_trained.pth'
!mkdir /content/Graphonomy/data/pretrained_model
os.system('cp "{0}" /content/Graphonomy/data/pretrained_model/'.format(pascal_pt))
os.system('cp "{0}" /content/Graphonomy/data/pretrained_model/'.format(cihp_pt))
os.system('cp "{0}" /content/Graphonomy/data/pretrained_model/'.format(universal_pt))

!sudo apt-get install unrar
#The datasets (CIHP & ATR) are available at google drive and baidu drive. 
os.system('cp "{0}" /content/Graphonomy/data/datasets/ATR/'.format(atr_path))
os.system('cp "{0}" /content/Graphonomy/data/datasets/CIHP_4w/'.format(cihp_path))

#And you also need to download the label with flipped. 

#Download cihp_flipped, unzip and store in data/datasets/CIHP_4w/.
%cd /content/Graphonomy/data/datasets/CIHP_4w
os.system('cp "{0}" /content/Graphonomy/data/datasets/CIHP_4w/'.format(cihp_flipped_path))
!unrar x Category_rev_ids.rar

#Download atr_flip, unzip and store in data/datasets/ATR/.
%cd /content/Graphonomy/data/datasets/ATR
os.system('cp "{0}" /content/Graphonomy/data/datasets/CIHP_4w/'.format(atr_flipped_path))
!unrar x SegmentationClassAug_rev.rar



In [0]:
#!cp "/content/gdrive/My Drive/Category_rev_ids.rar" /content/Graphonomy/data/datasets/CIHP_4w/



In [18]:
# load some sample images
%cd /content/Graphonomy/img
#too big
!wget -O 2190.jpg https://cdn-pics.pornhd.com/pornhd/header/category/2_phd_category_big_anal-porn.jpg
#too big
!wget -O 5008.jpg http://badmilfs.net/wp-content/uploads/2016/06/badmilfs-porn.jpg
#works
!wget -O 2008.jpg https://images.nubiles-porn.com/assets/common/images/tubeTourThumbs/147/523/147523/fullSizeCropped_147523.jpg?cacheBust=a0966056

#medium size
!wget -O 1111.jpg http://www.rushporn.com/screen/videos/images/horny-fucker-fucks-sexy-russian-girls-on-hd-quality-group-porn-1502750300.jpg
!wget -O 2222.jpg http://www.rexporn.com/static/tehreesome-squirting-porn.-alexis-fawx-and-her-stepdaughter-brittany-shae.jpg
!wget -O 3333.jpg https://www.vidtubeporn.com/fullimage/beautiful-porn-started-in-the-kitchen-and-continued-in-the-crib_1.jpg
%cd /content/Graphonomy

/content/Graphonomy/img
--2019-05-05 01:03:56--  https://cdn-pics.pornhd.com/pornhd/header/category/2_phd_category_big_anal-porn.jpg
Resolving cdn-pics.pornhd.com (cdn-pics.pornhd.com)... 64.210.147.100, 64.210.147.102, 64.210.147.104
Connecting to cdn-pics.pornhd.com (cdn-pics.pornhd.com)|64.210.147.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222592 (217K) [image/jpeg]
Saving to: ‘2190.jpg’

2190.jpg            100%[===================>] 217.38K  --.-KB/s    in 0.03s   

2019-05-05 01:03:56 (7.02 MB/s) - ‘2190.jpg’ saved [222592/222592]

--2019-05-05 01:03:57--  http://badmilfs.net/wp-content/uploads/2016/06/badmilfs-porn.jpg
Resolving badmilfs.net (badmilfs.net)... 5.149.248.193
Connecting to badmilfs.net (badmilfs.net)|5.149.248.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414272 (405K) [image/jpeg]
Saving to: ‘5008.jpg’

5008.jpg            100%[===================>] 404.56K   507KB/s    in 0.8s    

2019-05-05 01:03

In [21]:
import torch
torch.cuda.empty_cache()

# We provide a simply script to get the visualization result on the CIHP dataset using trained models as follows :
# Example of inference
%cd /content/Graphonomy
!python exp/inference/inference.py  \
--loadmodel ./data/pretrained_model/inference.pth \
--img_path ./img/3333.jpg \
--output_path ./img/ \
--output_name /test.jpg



/content/Graphonomy
Constructing DeepLabv3+ model...
Number of classes: 20
Output stride: 16
Number of Input Channels: 3
missing keys in state_dict: "{'xception_features.block17.rep.3.pointwise_bn.num_batches_tracked', 'decoder.1.atrous_convolution.pointwise_bn.num_batches_tracked', 'xception_features.block6.rep.5.depthwise_bn.num_batches_tracked', 'aspp4.atrous_convolution.pointwise_bn.num_batches_tracked', 'xception_features.block4.rep.1.pointwise_bn.num_batches_tracked', 'xception_features.block6.rep.1.pointwise_bn.num_batches_tracked', 'xception_features.block14.rep.1.pointwise_bn.num_batches_tracked', 'aspp3.atrous_convolution.depthwise_bn.num_batches_tracked', 'xception_features.block6.rep.3.pointwise_bn.num_batches_tracked', 'xception_features.block4.rep.1.depthwise_bn.num_batches_tracked', 'xception_features.block1.rep.2.depthwise_bn.num_batches_tracked', 'xception_features.block9.rep.1.pointwise_bn.num_batches_tracked', 'aspp3.atrous_convolution.pointwise_bn.num_batches_tracke

In [8]:
!ps
!kill 8644

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      6 ?        00:00:00 node
     26 ?        00:00:01 node
     36 ?        00:00:01 jupyter-noteboo
    124 ?        00:00:00 tail
    135 ?        00:00:06 python3
    177 ?        00:00:00 drive
    245 ?        00:00:07 drive
    257 ?        00:00:00 fusermount <defunct>
    301 ?        00:00:00 tail
    302 ?        00:00:00 grep
    401 ?        00:00:00 ps


KeyboardInterrupt: ignored